## Implementation of Hybrid ESN for Lorenz 96

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV


### Data Generation and Noising

### Ensemble Kalman Filtering

In [ ]:
import numpy as np

class EnKF:
    def __init__(self, ensemble, forecast_func, H, R):
        """
        Ensemble Kalman Filter implementation based on a forecast model.

        Parameters:
        - ensemble: numpy array of shape (N_ens, state_dim)
        - forecast_func: function to forecast next state (applied to each ensemble member)
        - H: observation matrix (obs_dim x state_dim)
        - R: observation noise covariance (obs_dim x obs_dim)
        """
        self.ensemble = ensemble
        self.forecast_func = forecast_func
        self.H = H
        self.R = R
        self.xEnKF = []
        self.x_ens = []

    def predict(self):
        """
        Propagates each ensemble member forward one step using the forecast function.

        Parameters:
        - None

        Returns:
        - None (updates self.ensemble in place)
        """
        for i in range(len(self.ensemble)):
            self.ensemble[i] = self.forecast_func(self.ensemble[i])

    def update(self, observation):
        """
        Applies the Kalman filter update step using the current observation.

        Parameters:
        - observation: numpy array of shape (obs_dim,), the current observed value

        Returns:
        - None (updates self.ensemble in place, and stores the ensemble mean and members)
        """
        N = len(self.ensemble)
        X = np.array(self.ensemble).T  # shape: (state_dim, N)
        x_mean = np.mean(X, axis=1, keepdims=True)
        X_prime = X - x_mean

        # Observation space
        HX = self.H @ X
        y_mean = np.mean(HX, axis=1, keepdims=True)
        Y_prime = HX - y_mean

        # Kalman gain
        P_xy = X_prime @ Y_prime.T / (N - 1)
        P_yy = Y_prime @ Y_prime.T / (N - 1) + self.R
        K = P_xy @ np.linalg.inv(P_yy)

        # Update ensemble
        for i in range(N):
            perturb = np.random.multivariate_normal(np.zeros(self.R.shape[0]), self.R)
            innovation = observation + perturb - HX[:, i]
            X[:, i] = X[:, i] + K @ innovation

        self.ensemble = X.T
        self.xEnKF.append(np.mean(self.ensemble, axis=0))
        self.x_ens.append(np.copy(self.ensemble))

    def get_results(self):
        """
        Retrieves the stored ensemble mean trajectory and full ensemble trajectories.

        Parameters:
        - None

        Returns:
        - xEnKF: numpy array of shape (timesteps, state_dim), ensemble mean at each timestep
        - x_ens: numpy array of shape (timesteps, N_ens, state_dim), ensemble members at each timestep
        """
        return np.array(self.xEnKF), np.array(self.x_ens)

### Hybrid ESN

### Evaluation Metrics